In [1]:
import numpy

In [170]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.9.0
GPU 사용 가능 여부: True


In [184]:
batch_size = 10
num_epochs = 10
learning_rate = 0.00000000000001

데이터 셋 경로 및 기본 설정

In [4]:
data_dir = './corona'

In [124]:
import csv
 
f = open( data_dir + '/Covid19InfState.csv','r')
rdr = csv.reader(f)
num = 0
x_data = []
y_data = []
for line in rdr:
    num += 1
    if num >= 62:
        print(line[10], line[3])
        x_data.append(float(line[10]))
        y_data.append(float(line[3]))

f.close()

75792 4212
90296 4812
108293 5328
124731 5766
142908 6284
158569 6767
169142 7134
179160 7382
191692 7513
203855 7755
217271 7869
230707 7979
243701 8086
251940 8162
259533 8236
269425 8320
279301 8413
291120 8565
301139 8652
311805 8799
317240 8897
324408 8961
333142 9037
333142 9037
343618 9137
350573 9241
361742 9332
371361 9478
379113 9583
381663 9661
393672 9786
404962 9887
413858 9976
413858 9976
424365 10062
434888 10156
441662 10237
447509 10284
456654 10331
468145 10384
479202 10423
487753 10450
496409 10480
500833 10512
505352 10537
512787 10564
519526 10591
524507 10613
532277 10635
541284 10653
546866 10661
551054 10674
558293 10683
565838 10694
573832 10702
579920 10708
587985 12801
585902 10718
585902 10718
589286 10728
592765 10738
599311 10752
605890 10761
611247 10765
614384 10774
619066 10780
622385 10793
625745 10801
631379 10804
635086 10806
640959 10810
645996 10822
650877 10840
653758 10874
657570 10909
664560 10936
676341 10962
690762 10991
706872 11018
722302 11

In [125]:
import numpy as np

x_data = np.array(x_data)
y_data = np.array(y_data)
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_data.shape) # x_data shape
print(len(x_data))  # x_data 길이
print(x_data[:5])   # 첫 다섯 개
print(x_train.shape)

print(y_data.shape) # y_data shape
print(len(y_data))  # y_data 길이
print(y_data[:5])   # 첫 다섯 개

(564,)
564
[ 75792.  90296. 108293. 124731. 142908.]
torch.Size([564])
(564,)
564
[4212. 4812. 5328. 5766. 6284.]


In [130]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [131]:
torch.manual_seed(1)

In [173]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
x_train = x_train.unsqueeze(1)
y_train = y_train.unsqueeze(1)


In [174]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [188]:
# 모델 초기화
model = MultivariateLinearRegressionModel()

optimizer = optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

nb_epochs = 100
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #  로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/100 Cost: 26747795931136.000000
Epoch   10/100 Cost: 300583552.000000
Epoch   20/100 Cost: 300583488.000000
Epoch   30/100 Cost: 300583488.000000
Epoch   40/100 Cost: 300583488.000000
Epoch   50/100 Cost: 300583488.000000
Epoch   60/100 Cost: 300583488.000000
Epoch   70/100 Cost: 300583488.000000
Epoch   80/100 Cost: 300583488.000000
Epoch   90/100 Cost: 300583488.000000
Epoch  100/100 Cost: 300583488.000000


In [193]:
number = float(input())
new_var =  torch.FloatTensor([[number]]) 
pred_y = model(new_var) # forward 연산
print('훈련 후 입력이 {}일 때의 예측값 :'.format(number), pred_y) 

560000
훈련 후 입력이 560000.0일 때의 예측값 : tensor([[8970.8330]], grad_fn=<AddmmBackward>)


테스트 함수

In [187]:
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=learning_rate)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train*W - b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
    epoch, nb_epochs, cost.item()


    ))

Epoch    0/20 Cost: 10403862528.000000
Epoch   10/20 Cost: 300582496.000000
Epoch   20/20 Cost: 300582464.000000


In [121]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

    
model = LinearRegressionModel()
hypothesis = model(x_train.unsqueeze(1))
print(hypothesis)

tensor([[  -36563.3945],
        [  -43560.3047],
        [  -52242.2773],
        [  -60172.1719],
        [  -68940.9844],
        [  -76496.0391],
        [  -81596.5859],
        [  -86429.3906],
        [  -92474.9844],
        [  -98342.5625],
        [ -104814.6094],
        [ -111296.3047],
        [ -117564.7656],
        [ -121539.3594],
        [ -125202.3203],
        [ -129974.3438],
        [ -134738.6406],
        [ -140440.2656],
        [ -145273.5625],
        [ -150418.9688],
        [ -153040.8750],
        [ -156498.8125],
        [ -160712.2031],
        [ -160712.2031],
        [ -165765.9531],
        [ -169121.1250],
        [ -174509.1875],
        [ -179149.5156],
        [ -182889.1719],
        [ -184119.3281],
        [ -189912.6250],
        [ -195359.0625],
        [ -199650.5938],
        [ -199650.5938],
        [ -204719.2969],
        [ -209795.7344],
        [ -213063.5938],
        [ -215884.2500],
        [ -220295.9219],
        [ -225839.3125],


In [119]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train.unsqueeze(1))

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train.unsqueeze(1))

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 149.356628 Accuracy 0.00%
Epoch   10/1000 Cost: 149.356628 Accuracy 0.00%
Epoch   20/1000 Cost: 149.356628 Accuracy 0.00%
Epoch   30/1000 Cost: 149.356628 Accuracy 0.00%
Epoch   40/1000 Cost: 149.356628 Accuracy 0.00%
Epoch   50/1000 Cost: 149.356628 Accuracy 0.00%
Epoch   60/1000 Cost: 149.356628 Accuracy 0.00%
Epoch   70/1000 Cost: 149.356628 Accuracy 0.00%
Epoch   80/1000 Cost: 149.356628 Accuracy 0.00%
Epoch   90/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  100/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  110/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  120/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  130/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  140/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  150/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  160/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  170/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  180/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  190/1000 Cost: 149.356628 Accuracy 0.00%
Epoch  200/1000 Cost: 149.356628 Accurac